In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import warnings

warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", 50)

## Data Loading

In [ ]:
bucket = 's3://ads508-imdb-data'

titles = pd.read_csv(f'{bucket}/ImdbTitleBasics.csv', low_memory=False)
plots = pd.read_csv(f'{bucket}/movies_plots.csv', low_memory=False)
netflix = pd.read_csv(f'{bucket}/NetFlix.csv', low_memory=False)

In [ ]:
def fuzzy_match(x):
    
    global step
    
    results = process.extractOne(x['IMDB Title'], netflix['Netflix Title'].tolist(), scorer=fuzz.ratio)
    
    print('{}/{}'.format(step, netflix.shape[0]), flush=True, end='\r')
    
    step += 1
    
    return results[0], results[1]

In [ ]:
titles['imdb_id'] = titles['tconst'].replace('tt', '', regex=True).astype(int)

titles = titles.merge(plots[['imdbID', 'plot', 'fullplot', 'country', 'awards']], how='inner', left_on='imdb_id', right_on='imdbID')\
                   .drop(columns=['imdb_id'])\
                   .drop_duplicates()

netflix.rename(columns={'title':'Netflix Title'}, inplace=True)

In [ ]:
match_list = titles[['originalTitle']].rename(columns={'originalTitle':'IMDB Title'}).drop_duplicates()

In [ ]:
# step is for printing purpose
step = 1
match_list[['Matched Netflix Title', 'Match Score']] = match_list.apply(fuzzy_match, axis=1, result_type="expand")

In [ ]:
# Only keep those titles with 90 or higher match score
match_list = match_list[match_list['Match Score']>=90]